In [ ]:
!git clone https://github.com/NINAnor/ecoVAD.git

In [ ]:
!pip install audiomentations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from yaml import FullLoader
import yaml
import glob
import os

In [ ]:
current_dir = os.getcwd()

In [ ]:
with open(current_dir + "/ecoVAD/config_training.yaml") as f:
    cfg = yaml.load(f, Loader=FullLoader)

In [ ]:
audio_path = current_dir + '/Sound_Scape_Data'
speech_dir = current_dir + '/LibSpeechData'
noise_dir = current_dir + '/Anthropogenic_BirdCLEF'
audio_out_dir =current_dir + '/Synthetic_Dataset'
checkpoint_file = current_dir +'/checkpoint.txt'
os.chdir(current_dir+"/ecoVAD")

In [ ]:
# Partial-Code from Cretois et al. (2022)
from VAD_algorithms.ecovad.make_data import preprocess_file, save_processed_arrays
def read_processed_files(filepath):
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            processed_files = [line.strip() for line in f.readlines()]
    else:
        processed_files = []
    return processed_files

def save_processed_file(filepath, file):
    with open(filepath, 'a') as f:
        f.write(file + '\n')

list_audio_files = glob.glob(audio_path+ "/*")
processed_files = read_processed_files(checkpoint_file)


for file in list_audio_files:
    if file not in processed_files:
        try:
            processed_arr, sr = preprocess_file(file, 
                            cfg["LENGTH_SEGMENTS"], 
                            overlap=0, 
                            min_length=cfg["LENGTH_SEGMENTS"],
                            speech_dir=speech_dir,
                            noise_dir=noise_dir,
                            proba_speech=cfg["PROBA_SPEECH"],
                            proba_noise_speech=cfg["PROBA_NOISE_WHEN_SPEECH"],
                            proba_noise_nospeech=cfg["PROBA_NOISE_WHEN_NO_SPEECH"])
            save_processed_arrays(file, audio_out_dir, processed_arr, sr)
            save_processed_file(checkpoint_file, file)
        except AssertionError:
            print(f"Could not find noise files for {file}")